# Asset Selling Driver Script
This script is used to run the Asset Selling Model from Chapter 2 of W.B.Powell's book "Sequential decision analytics and modeling". It is based on the initial version from the repository referred in the book (https://github.com/wbpowell328/stochastic-optimization).

The script reads in the parameters from an Excel file and then runs the model.

### Read in parameters from Excel file

In [21]:
%load_ext autoreload
%autoreload 2

from collections import namedtuple
import time
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "seaborn"

from AssetSellingModel import AssetSellingModel
from AssetSellingPolicy import AssetSellingPolicy

# read in policy parameters from an Excel spreadsheet, "asset_selling_policy_parameters.xlsx"
sheet1 = pd.read_excel("asset_selling_policy_parameters.xlsx", sheet_name="Sheet1")
# theta values for the three policies
params = zip(sheet1["param1"], sheet1["param2"])
param_list = list(params)
sheet2 = pd.read_excel("asset_selling_policy_parameters.xlsx", sheet_name="Sheet2")
sheet3 = pd.read_excel("asset_selling_policy_parameters.xlsx", sheet_name="Sheet3")
biasdf = pd.read_excel(
    "asset_selling_policy_parameters.xlsx", sheet_name="Sheet4", index_col=0
)

policy_selected = sheet3["Policy"][0]
T = sheet3["TimeHorizon"][0]
initPrice = sheet3["InitialPrice"][0]
initBias = sheet3["InitialBias"][0]
grid_search = False if sheet3["Evaluation"][0] == "single" else True

exog_params = {
    "UpStep": sheet3["UpStep"][0],
    "DownStep": sheet3["DownStep"][0],
    "Variance": sheet3["Variance"][0],
    "biasdf": biasdf,
}

nIterations = sheet3["Iterations"][0]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Run the model

In [22]:
# initialize the model and the policy
policy_names = ["sell_low", "high_low", "track"]
state_names = ["price", "price_smoothed", "resource", "bias"]
init_state = {
    "price": initPrice,
    "price_smoothed": initPrice,
    "resource": 1,
    "bias": initBias,
}
decision_names = ["sell", "hold"]

M = AssetSellingModel(state_names, decision_names, init_state, exog_params, T)
P = AssetSellingPolicy(M, policy_names)
t = 0

# make a policy_info dict object to pass policy-specific parameters
policy_info = {
    "sell_low": param_list[0],
    "high_low": param_list[1],
    "track": param_list[2],
}

start = time.time()
print(
    "Selected policy {}, time horizon {}, initial price {} and number of iterations {}".format(
        policy_selected, T, initPrice, nIterations
    )
)

# Evaluate one of the policies for one value of theta
if grid_search == False:
    contribution_iterations = []
    state_histories = []
    for i in range(nIterations):
        contribution, t_stop, state_history = P.run_policy(
            policy_info, policy_selected, t
        )
        contribution_iterations.append((contribution, t_stop))
        state_history["iteration"] = i
        state_histories.append(state_history)

    contribution_iterations = pd.DataFrame(
        contribution_iterations, columns=["objective", "stopping time"]
    )
    state_histories = pd.concat(state_histories)

    print("Contribution per iteration: ")
    print(contribution_iterations)

    fig = make_subplots(rows=2, cols=2, subplot_titles=["objective", "stopping time"])

    fig.add_trace(
        go.Histogram(x=contribution_iterations["objective"]),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Histogram(x=contribution_iterations["stopping time"]), row=1, col=2
    )

    fig.add_trace(
        go.Scatter(
            x=contribution_iterations["stopping time"],
            y=contribution_iterations["objective"],
            mode="markers",
        ),
        row=2,
        col=1,
    )
    fig["layout"]["xaxis3"]["title"] = "Stopping time"
    fig["layout"]["yaxis3"]["title"] = "Objective"

    fig.update_layout(
        height=600,
        width=800,
        title_text="Distribution of objectives and stopping times",
    )
    fig.show()

else:
    # Run a grid-search procedure for the selected policy

    # Get the theta values to carry out a full grid search
    theta_values, theta_high_values, theta_low_values = P.grid_search_theta_values(
        policy_selected,
        sheet2["low_min"][0],
        sheet2["low_max"][0],
        sheet2["high_min"][0],
        sheet2["high_max"][0],
        sheet2["track_min"][0],
        sheet2["track_max"][0],
        sheet2["increment_size"][0],
    )
    # use those theta values to calculate corresponding contribution values

    contribution_iterations = [
        P.vary_theta(policy_info, policy_selected, t, theta_values)
        for ite in list(range(nIterations))
    ]

    contribution_iterations = np.array(contribution_iterations)
    contribution_avgs = contribution_iterations.mean(axis=0)

    if policy_selected == "high_low":
        # plot average contributions of theta values on a heat map
        contribution_matrix = np.reshape(
            contribution_avgs[:, 0], (len(theta_low_values), -1)
        )
        fig = px.imshow(
            contribution_matrix,
            labels={"x": "theta_high", "y": "theta_low"},
            x=theta_high_values,
            y=theta_low_values,
        )
        fig.show()
    else:
        # plot average contributions of theta values in a line graph
        fig = px.line(x=theta_low_values, y=contribution_avgs[:, 0])
        fig.show()

end = time.time()
print("{} secs".format(end - start))

Selected policy high_low, time horizon 100, initial price 16 and number of iterations 10
Contribution per iteration: 
   objective  stopping time
0  38.490092            100
1   6.492939            100
2   3.638744            100
3  66.567903            100
4   1.105618            100
5  40.657028            100
6  87.364135            100
7   2.057562            100
8  39.824136            100
9   3.393173            100


0.8865141868591309 secs


In [23]:
state_histories

,t,price,price_smoothed,resource,bias,iteration
0,0,16.000000,16.000000,1,Up,0
1,1,14.920032,15.244022,1,Up,0
2,2,17.801161,17.034020,1,Up,0
3,3,16.196802,16.447967,1,Up,0
4,4,16.564317,16.529412,1,Up,0
...,...,...,...,...,...,...
96,96,0.000000,0.001329,1,Down,9
97,97,0.593088,0.415560,1,Down,9
98,98,0.000000,0.124668,1,Down,9
99,99,3.393173,2.412622,1,Neutral,9


In [25]:
px.line(
    data_frame=state_histories,
    x="t",
    y=["price", "price_smoothed"],
    facet_row="iteration",
    height=1200,
    width=400,
)
# px.line(data_frame=state_histories, x="t", y="price", facet_row="iteration", height=1200)